In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import keras

In [4]:
df = pd.read_csv('../KZ_project_setup/outputs/data_ind/BTC-USD/BTC-USD_df_2y_1h.csv')
df = df.drop(columns=['candlestick_pattern'], axis=1)
df['Datetime'] = pd.to_datetime(df['Datetime'])
df_btc = df.iloc[:-1, :].copy()
df_btc = df_btc.dropna()

In [6]:
result_test = df.iloc[-1:,:].copy()
#result_test = result_test.drop(columns=['d_r'], axis=1)
result_test

,Datetime,Open,High,Low,Close,Adj Close,Volume,sma_7,sma_8,sma_9,...,natr_19,natr_20,natr_30,natr_40,natr_50,natr_99,natr_200,candle_label,d_r,feature_label
8730,2022-10-16 18:00:00+00:00,19141.535156,19141.535156,19134.046875,19136.228516,19136.228516,973824,19139.791295,19140.947754,19139.801432,...,0.185093,0.190779,0.240078,0.273821,0.296245,0.347953,0.409841,2.0,-0.000284,NaN


In [7]:
df_btc = df_btc.reset_index()
df_btc = df_btc.drop(columns=['index'], axis=1)
df_btc['feature_label'].value_counts()

0.0    3797
1.0    3739
Name: feature_label, dtype: int64

In [8]:
col_list = df_btc.columns.to_list()
y_col = col_list.pop(-1)

In [9]:
X = df_btc[col_list[1:]]
y = df_btc[y_col]

In [12]:
df_btc.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume,sma_7,sma_8,sma_9,...,natr_19,natr_20,natr_30,natr_40,natr_50,natr_99,natr_200,candle_label,d_r,feature_label
0,2021-12-06 05:00:00+00:00,49047.976562,49124.144531,48809.332031,48838.476562,48838.476562,0,49026.256696,49053.649902,49078.984809,...,1.166766,1.178520,1.252772,1.278089,1.281560,1.234162,1.162978,-1.0,-0.004550,0.0
1,2021-12-06 06:00:00+00:00,48800.183594,48811.308594,48150.199219,48150.199219,48150.199219,0,48851.533482,48916.749512,48953.266493,...,1.196391,1.207070,1.275971,1.299685,1.302471,1.253598,1.180852,-1.0,-0.014093,1.0
2,2021-12-06 07:00:00+00:00,48177.371094,48375.832031,47873.671875,48375.832031,48375.832031,169101312,48789.851562,48792.070801,48856.647569,...,1.182770,1.193270,1.262287,1.287234,1.291229,1.245632,1.174657,2.0,0.004686,0.0
3,2021-12-06 08:00:00+00:00,48445.027344,49321.496094,47924.988281,48261.097656,48261.097656,1006387200,48698.352679,48723.757324,48733.073785,...,1.275480,1.280984,1.319567,1.330378,1.326285,1.265210,1.186031,-1.0,-0.002372,0.0
4,2021-12-06 09:00:00+00:00,48230.355469,48292.523438,47365.312500,47365.312500,47365.312500,64221184,48458.920759,48531.722656,48572.819010,...,1.334233,1.337829,1.364958,1.370589,1.363493,1.295890,1.212207,-1.0,-0.018561,1.0


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [11]:
DTC = DecisionTreeClassifier(criterion='gini',
                    max_depth=5,
                    random_state=42)
DTC.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [12]:
from sklearn.metrics import classification_report

In [13]:
print(classification_report(y_test, DTC.predict(X_test[X_train.columns])))

              precision    recall  f1-score   support

         0.0       0.54      0.67      0.60       783
         1.0       0.52      0.39      0.45       725

    accuracy                           0.53      1508
   macro avg       0.53      0.53      0.52      1508
weighted avg       0.53      0.53      0.52      1508



In [14]:
import eli5
from eli5.sklearn import PermutationImportance

In [15]:
perm = PermutationImportance(DTC, random_state=1).fit(X_test[X_train.columns], y_test)
eli5.show_weights(perm, feature_names = X_test[X_train.columns].columns.tolist())

Weight,Feature
0.0183 ± 0.0192,wllr_8
0.0090 ± 0.0111,natr_8
0.0073 ± 0.0047,d_r
0.0009 ± 0.0014,dmi_up_200
0.0007 ± 0.0008,natr_20
0.0007 ± 0.0000,wllr_18
0.0007 ± 0.0000,trima_99
0.0005 ± 0.0005,tema_30
0.0004 ± 0.0029,atr_11
0 ± 0.0000,kama_50


In [16]:
col_list = ['wllr_20', 'macdsignal', 'dmi_down_30', 'atr_15', 'cci_10', 'atr_40', 't3_50']

In [17]:
X = df_btc[col_list]
y = df_btc[y_col]

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train)

print(classification_report(y_test, DTC.predict(X_test[X_train.columns])))

              precision    recall  f1-score   support

         0.0       0.48      0.50      0.49       358
         1.0       0.53      0.51      0.52       396

    accuracy                           0.50       754
   macro avg       0.50      0.50      0.50       754
weighted avg       0.50      0.50      0.50       754

